<a href="https://colab.research.google.com/github/sagarikajadon13/Grammar-Error-Detection/blob/main/3_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 30.9 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import re
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import transformers
from transformers import BertConfig, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
train= pd.read_csv('/content/drive/MyDrive/SHL NLP task/train_data.csv')
val= pd.read_csv('/content/drive/MyDrive/SHL NLP task/val_data.csv')
test= pd.read_csv('/content/drive/MyDrive/SHL NLP task/test_data.csv')

In [ ]:
train

,input,labels
0,I am reading score of Mahler is Symphony No . .,0
1,I am not interested in cars or electric applia...,1
2,This is my homework for my English class .,0
3,"In comparison , Canada is catches increased an...",0
4,"Fortunately , my older sister is friend is a d...",1
...,...,...
19993,More than a hundred thousand people are suppor...,1
19994,The reason why we chose this subject a few day...,1
19995,When you buy CBS is Criminal Minds DVD in Japa...,1
19996,but it is dizzying for me T T,1


In [ ]:
to_drop= (train.loc[train['input']== ' '])

In [ ]:
to_drop.index

Int64Index([  254,   259,   512,   629,  1033,  1129,  1529,  2149,  2558,
             2879,  2938,  3537,  3714,  3958,  4046,  4068,  4083,  4153,
             4399,  4493,  5128,  5165,  5318,  5622,  5976,  6021,  6313,
             6317,  6470,  6986,  7003,  7216,  7585,  7779,  7879,  8013,
             8136,  8416,  8748,  9058,  9122,  9733,  9872, 10619, 11027,
            11762, 11923, 11971, 12334, 12564, 12611, 12869, 12906, 13053,
            13326, 14287, 14292, 14308, 14773, 14836, 15260, 15407, 15744,
            15827, 15864, 15959, 16912, 17040, 17696, 17786, 18428, 18611,
            19176, 19376, 19883],
           dtype='int64')

In [ ]:
train= train.drop(to_drop.index, axis= 0).reset_index(drop= True)

In [ ]:
train.tail()

,input,labels
19918,More than a hundred thousand people are suppor...,1
19919,The reason why we chose this subject a few day...,1
19920,When you buy CBS is Criminal Minds DVD in Japa...,1
19921,but it is dizzying for me T T,1
19922,in several times .,0


In [ ]:
train['input_len']= train['input'].apply(lambda x: len(x.split(' ')))

In [ ]:
train[train['input_len']== 319]

,input,labels,input_len
19524,Corporate Governance To check whether the meet...,0,319


In [ ]:
text_319= train.loc[19524].input

In [ ]:
text_319= text_319.split('.')

In [ ]:
text_319

['Corporate Governance To check whether the meeting records of the Board of Directors and Special Committee were complete and signed by directors who attended the meetings ',
 ' To check the contents , deadline and signatures of the formal written authorization of shareholder representatives ',
 ' To check whether scopes of authority were written in letters of authorization by the directors , the letters were signed by directors and the meeting records took down the trustees who attended the meetings ',
 ' To read all the formal written meeting records and to check whether the frequency of arranging the Board meetings complies with articles of incorporation ',
 ' To check the meeting records , resolutions and other related contents of Board of Supervisors and to check whether the frequency of arranging the meeting complies with articles of incorporation ',
 ' To check the meeting records , resolutions and other related contents of the three Special Committees and whether the frequency 

In [ ]:
df= pd.DataFrame({'input': text_319,
                  'labels': [1]* len(text_319)})

In [ ]:
train= train.append(df, ignore_index= True)

In [ ]:
train= train.drop(19524, axis= 0).reset_index(drop= True)

In [ ]:
tokenizer= BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
tokenizer('I am going to visit Busan this coming weekend .')['input_ids']

[101, 146, 1821, 1280, 1106, 3143, 8947, 1389, 1142, 1909, 5138, 119, 102]

In [ ]:
tokenizer.convert_ids_to_tokens([101, 146, 1821, 1280, 1106, 3143, 8947, 1389, 1142, 1909, 5138, 119, 102])

['[CLS]',
 'I',
 'am',
 'going',
 'to',
 'visit',
 'Bus',
 '##an',
 'this',
 'coming',
 'weekend',
 '.',
 '[SEP]']

In [ ]:
class GEDdataset:
  def __init__(self, raw_inputs, labels, tokenizer, max_length):
    self.raw_inputs= raw_inputs
    self.labels= labels
    self.tokenizer= tokenizer
    self.max_length= max_length

  def __len__(self):
    return len(self.raw_inputs)

  def __getitem__(self, idx):
    text= self.raw_inputs[idx]
    label= self.labels[idx]

    encoding= self.tokenizer.encode_plus(text,
                                         add_special_tokens= True,
                                         max_length= self.max_length,
                                         return_token_type_ids= False,
                                         padding= 'max_length',
                                         truncation= True,
                                         return_attention_mask= True,
                                         return_tensors= 'pt')
    return {
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'label': torch.tensor(label, dtype= torch.long)
    }

In [ ]:
model= BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels= 2).to(device)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
def train_model(dataloader, model, optimizer):
    size= len(dataloader.dataset)
    n_batches= len(dataloader)
    train_loss= 0
    
    model.train()
    for item in dataloader:
      input_ids, attention_mask, label= item['input_ids'].to(device), item['attention_mask'].to(device), item['label'].to(device)

      loss, logits= model(input_ids= input_ids,
                    attention_mask= attention_mask,
                    output_attentions= False,
                    output_hidden_states= False,
                    labels= label, 
                    return_dict = False)
   
      optimizer.zero_grad()
      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      
      loss= loss.item()
      train_loss += loss
        
    train_loss_avg= train_loss/n_batches
    print(f"avg. train loss: {train_loss_avg}")
    return train_loss_avg

In [ ]:
def val_model(dataloader, model):
    size= len(dataloader.dataset)
    n_batches= len(dataloader)
    test_loss= 0

    model.eval()
    with torch.no_grad():
      for item in dataloader:
        input_ids, attention_mask, label= item['input_ids'].to(device), item['attention_mask'].to(device), item['label'].to(device)

        loss, logits= model(input_ids= input_ids,
                      attention_mask= attention_mask,
                      output_attentions= False,
                      output_hidden_states= False,
                      labels= label,
                      return_dict = False)
        
        test_loss += loss

    test_loss /= n_batches
    print(f"avg test loss : {test_loss}")
    return test_loss

In [ ]:
def predict_model(dataloader, model):
    model.eval()
    y_pred= np.array([])
    
    with torch.no_grad():
      for item in dataloader:
        input_ids, attention_mask, label= item['input_ids'].to(device), item['attention_mask'].to(device), item['label'].to(device)

        (logits)= model(input_ids= input_ids,
                      attention_mask= attention_mask,
                      output_attentions= False,
                      output_hidden_states= False,
                      return_dict= False
                      )
        
        logits= logits[0].detach().cpu().numpy()
        preds= np.argmax(logits, axis= 1)
        y_pred= np.concatenate((y_pred, preds))
    return y_pred

In [ ]:
train_inputs= train.input.values
train_labels= train.labels.values

val_inputs= val.input.values
val_labels= val.labels.values

test_inputs= test.input.values
test_labels= np.zeros(test_inputs.shape[0])

In [ ]:
EPOCHS= 3
TRAIN_BATCHSIZE= 32
TEST_BATCHSIZE= 32
LEARNING_RATE= 1e-3

In [ ]:
train_dataset= GEDdataset(train_inputs, train_labels, tokenizer, max_length= 15)
val_dataset= GEDdataset(val_inputs, val_labels, tokenizer, max_length= 15)
test_dataset= GEDdataset(test_inputs, test_labels, tokenizer, max_length= 15)

In [ ]:
train_loader= DataLoader(train_dataset, TRAIN_BATCHSIZE)
val_loader= DataLoader(val_dataset, TEST_BATCHSIZE)
test_loader= DataLoader(test_dataset, TEST_BATCHSIZE)

In [ ]:
optimizer= AdamW(model.parameters(), lr= LEARNING_RATE)
scheduler= get_linear_schedule_with_warmup(optimizer, num_warmup_steps= 0, num_training_steps= len(train_loader)*EPOCHS)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
best_val_loss= float('inf')
train_loss= []
val_loss= []

for i in range(EPOCHS):
  avg_train_loss= train_model(train_loader, model, optimizer)
  avg_val_loss= val_model(val_loader, model)

  train_loss.append(avg_train_loss)
  val_loss.append(avg_val_loss)

  if avg_val_loss< best_val_loss:
    best_val_loss= avg_val_loss
    filename= 'bert.pth'
    torch.save(model.state_dict(), filename)

  scheduler.step()

avg. train loss: 0.740640896568329
avg test loss : 0.7426131367683411
avg. train loss: 0.731209854731399
avg test loss : 0.7094506025314331
avg. train loss: 0.7302647861584806
avg test loss : 0.7026706337928772


In [ ]:
model.load_state_dict(torch.load('bert.pth', map_location= device))

<All keys matched successfully>

In [ ]:
val_preds= predict_model(val_loader, model)
f1_score(val_labels, val_preds)

0.6666666666666666

In [ ]:
test_preds= predict_model(test_loader, model)

In [ ]:
test_preds.shape

(9972,)

In [ ]:
test_inputs= test['input']

In [ ]:
sub= pd.DataFrame({'input': test_inputs,
                   'labels': test_preds})

In [ ]:
sub.to_csv('submission_bert.csv', index= False)